In [55]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#Import the preprocessing class for cleaning
from preprocess_class import preprocess_charity_data as pcd

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Optimization Attempt #1
### Execute further feature reduction and binning

In [38]:
data_2_clean = pcd(application_df)

In [39]:
cleaning_df = data_2_clean.drop_columns()

Choose the columns from the following:
Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')
Enter the list items as listed separated by spaces : EIN NAME SPECIAL_CONSIDERATIONS STATUS
The list of columns to be dropped is: ['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS']


In [40]:
cleaning_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [41]:
data_2_clean = pcd(cleaning_df)
clean_df = data_2_clean.reduce_classification(cutoff=1000)

Reducing the number of Classification Types in df.CLASSIFICATION
Cutoff value = 1000


In [42]:
clean_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [43]:
data_2_clean = pcd(clean_df)
clean_df = data_2_clean.reduce_affiliation(cutoff=1000)
clean_df.AFFILIATION.value_counts()

Reducing the number of Classification Types in df.AFFILIATION
Cutoff value = 1000


Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [44]:
data_2_clean = pcd(clean_df)
clean_df = data_2_clean.ask_amt_binning()

Binning the AM_ASK column


In [45]:
clean_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL,Amt_Binned
0,T10,Independent,C1000,ProductDev,Association,0,1,A
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1,B
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0,A
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1,B
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1,B


In [46]:
data_2_clean = pcd(clean_df)
clean_df = data_2_clean.reduce_app_types()

Reducing the number of Applitcation Types in df.APPLICATION_TYPE
Cutoff value = 500


In [47]:
clean_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL,Amt_Binned
0,T10,Independent,C1000,ProductDev,Association,0,1,A
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1,B
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0,A
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1,B
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1,B
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,0,A
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,0,A
34296,T3,CompanySponsored,C2000,Preservation,Association,0,0,A
34297,T5,Independent,C3000,ProductDev,Association,0,1,A


In [48]:
# Generate our categorical variable lists
cat_list = list(clean_df.select_dtypes(include=['object', 'category']))
cat_list

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'Amt_Binned']

In [49]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
    
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(clean_df[cat_list]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(cat_list)

# Merge one-hot encoded features and drop the originals
model_data_df = clean_df.merge(encode_df,left_index=True, right_index=True)
model_data_df = model_data_df.drop(cat_list,1)

C:\Users\jonat\AppData\Local\Temp\ipykernel_15988\661558683.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  model_data_df = model_data_df.drop(cat_list,1)


In [50]:
model_data_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,Amt_Binned_A,Amt_Binned_B,Amt_Binned_C
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [83]:
# Split our preprocessed data into our features and target arrays
y = model_data_df.IS_SUCCESSFUL.values
X = model_data_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, shuffle=True)

C:\Users\jonat\AppData\Local\Temp\ipykernel_15988\316117081.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = model_data_df.drop(["IS_SUCCESSFUL"],1).values


In [84]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Use the same hyperparameters from the previous model to test if intensive feature reduction improves model performance

In [85]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, 
        input_dim=number_input_features, 
        activation="relu"
    )
)
# Second hidden layer
nn1.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, 
    activation="relu"
))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 80)                3200      
                                                                 
 dense_21 (Dense)            (None, 30)                2430      
                                                                 
 dense_22 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,661
Trainable params: 5,661
Non-trainable params: 0
_________________________________________________________________


In [86]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [87]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_attempt1/",exist_ok=True)
checkpoint_path = "checkpoints_attempt1/weights.{epoch:02d}.hdf5"

In [88]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5686 - accuracy: 0.7215
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5537 - accuracy: 0.7302
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7303
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7310
Epoch 5/150
793/804 [============================>.] - ETA: 0s - loss: 0.5479 - accuracy: 0.7306
Epoch 5: saving model to checkpoints_attempt1\weights.05.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7305
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7320
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7324
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7313
Epoch 9/150
804/804 [========================

804/804 [==============================] - 1s 2ms/step - loss: 0.5337 - accuracy: 0.7411
Epoch 63/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5330 - accuracy: 0.7395
Epoch 64/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5334 - accuracy: 0.7411
Epoch 65/150
802/804 [============================>.] - ETA: 0s - loss: 0.5333 - accuracy: 0.7404
Epoch 65: saving model to checkpoints_attempt1\weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7404
Epoch 66/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5331 - accuracy: 0.7403
Epoch 67/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5327 - accuracy: 0.7415
Epoch 68/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5326 - accuracy: 0.7398
Epoch 69/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5328 - accuracy: 0.7407
Epoch 70/150
794/804 [===========================

804/804 [==============================] - 1s 2ms/step - loss: 0.5298 - accuracy: 0.7413
Epoch 125/150
787/804 [============================>.] - ETA: 0s - loss: 0.5297 - accuracy: 0.7417
Epoch 125: saving model to checkpoints_attempt1\weights.125.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5301 - accuracy: 0.7415
Epoch 126/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5298 - accuracy: 0.7410
Epoch 127/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5299 - accuracy: 0.7414
Epoch 128/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5298 - accuracy: 0.7405
Epoch 129/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5298 - accuracy: 0.7414
Epoch 130/150
800/804 [============================>.] - ETA: 0s - loss: 0.5298 - accuracy: 0.7410
Epoch 130: saving model to checkpoints_attempt1\weights.130.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5300 - accuracy: 

In [89]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5640 - accuracy: 0.7262 - 418ms/epoch - 2ms/step
Loss: 0.5639626979827881, Accuracy: 0.7261807322502136


In [90]:
# Export the model to HDF5 file
nn1.save("Attempt1.h5")

## Attempt 2: Add another hidden layer

In [91]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 30

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, 
        input_dim=number_input_features, 
        activation="relu"
    )
)
# Second hidden layer
nn2.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, 
    activation="relu"
))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, 
    activation="relu"
))


# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 80)                3200      
                                                                 
 dense_24 (Dense)            (None, 30)                2430      
                                                                 
 dense_25 (Dense)            (None, 30)                930       
                                                                 
 dense_26 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,591
Trainable params: 6,591
Non-trainable params: 0
_________________________________________________________________


In [92]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [93]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_attempt2/",exist_ok=True)
checkpoint_path = "checkpoints_attempt2/weights.{epoch:02d}.hdf5"

In [94]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5701 - accuracy: 0.7181
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7279
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7313
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7314
Epoch 5/150
786/804 [============================>.] - ETA: 0s - loss: 0.5471 - accuracy: 0.7334
Epoch 5: saving model to checkpoints_attempt2\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7337
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7324
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7342
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7331
Epoch 9/150
804/804 [========================

804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7405
Epoch 63/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7411
Epoch 64/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5316 - accuracy: 0.7401
Epoch 65/150
781/804 [============================>.] - ETA: 0s - loss: 0.5325 - accuracy: 0.7400
Epoch 65: saving model to checkpoints_attempt2\weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5319 - accuracy: 0.7405
Epoch 66/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7407
Epoch 67/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7413
Epoch 68/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5314 - accuracy: 0.7411
Epoch 69/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5312 - accuracy: 0.7403
Epoch 70/150
800/804 [===========================

804/804 [==============================] - 2s 2ms/step - loss: 0.5283 - accuracy: 0.7420
Epoch 125/150
798/804 [============================>.] - ETA: 0s - loss: 0.5278 - accuracy: 0.7422
Epoch 125: saving model to checkpoints_attempt2\weights.125.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5278 - accuracy: 0.7423
Epoch 126/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5282 - accuracy: 0.7423
Epoch 127/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5283 - accuracy: 0.7423
Epoch 128/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5280 - accuracy: 0.7423
Epoch 129/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5276 - accuracy: 0.7422
Epoch 130/150
792/804 [============================>.] - ETA: 0s - loss: 0.5281 - accuracy: 0.7424
Epoch 130: saving model to checkpoints_attempt2\weights.130.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5279 - accuracy: 

In [95]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5708 - accuracy: 0.7266 - 435ms/epoch - 2ms/step
Loss: 0.5707940459251404, Accuracy: 0.7266472578048706


In [96]:
# Export the model to HDF5 file
nn2.save("Attempt2.h5")

## Attempt 3: Change the number of nodes and go back to two hidden layers

In [97]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = number_input_features*3
hidden_nodes_layer2 = hidden_nodes_layer1/2

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, 
        input_dim=number_input_features, 
        activation="relu"
    )
)
# Second hidden layer
nn3.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, 
    activation="relu"
))



# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 117)               4680      
                                                                 
 dense_28 (Dense)            (None, 58)                6844      
                                                                 
 dense_29 (Dense)            (None, 1)                 59        
                                                                 
Total params: 11,583
Trainable params: 11,583
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [99]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_attempt3/",exist_ok=True)
checkpoint_path = "checkpoints_attempt3/weights.{epoch:02d}.hdf5"

In [100]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5664 - accuracy: 0.7204
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7268
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7285
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7327
Epoch 5/150
800/804 [============================>.] - ETA: 0s - loss: 0.5475 - accuracy: 0.7323
Epoch 5: saving model to checkpoints_attempt3\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7324
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7336
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7333
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7322
Epoch 9/150
804/804 [========================

804/804 [==============================] - 2s 2ms/step - loss: 0.5316 - accuracy: 0.7397
Epoch 63/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7400
Epoch 64/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7408
Epoch 65/150
780/804 [============================>.] - ETA: 0s - loss: 0.5315 - accuracy: 0.7407
Epoch 65: saving model to checkpoints_attempt3\weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7405
Epoch 66/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5311 - accuracy: 0.7404
Epoch 67/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5314 - accuracy: 0.7410
Epoch 68/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5312 - accuracy: 0.7413
Epoch 69/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7411
Epoch 70/150
786/804 [===========================

804/804 [==============================] - 2s 2ms/step - loss: 0.5285 - accuracy: 0.7411
Epoch 125/150
782/804 [============================>.] - ETA: 0s - loss: 0.5290 - accuracy: 0.7418
Epoch 125: saving model to checkpoints_attempt3\weights.125.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5286 - accuracy: 0.7419
Epoch 126/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5280 - accuracy: 0.7414
Epoch 127/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5289 - accuracy: 0.7423
Epoch 128/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5287 - accuracy: 0.7410
Epoch 129/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5284 - accuracy: 0.7426
Epoch 130/150
785/804 [============================>.] - ETA: 0s - loss: 0.5280 - accuracy: 0.7416
Epoch 130: saving model to checkpoints_attempt3\weights.130.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5285 - accuracy: 

In [101]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5753 - accuracy: 0.7280 - 436ms/epoch - 2ms/step
Loss: 0.5752872824668884, Accuracy: 0.7280466556549072


In [102]:
# Export the model to HDF5 file
nn3.save("Attempt3.h5")

## Attempt 4: Change the change the activation function of the hidden layers

In [104]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = number_input_features*3
hidden_nodes_layer2 = hidden_nodes_layer1/2

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, 
        input_dim=number_input_features, 
        activation="tanh"
    )
)
# Second hidden layer
nn4.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, 
    activation="tanh"
))



# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 117)               4680      
                                                                 
 dense_34 (Dense)            (None, 58)                6844      
                                                                 
 dense_35 (Dense)            (None, 1)                 59        
                                                                 
Total params: 11,583
Trainable params: 11,583
Non-trainable params: 0
_________________________________________________________________


In [105]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [106]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_attempt4/",exist_ok=True)
checkpoint_path = "checkpoints_attempt4/weights.{epoch:02d}.hdf5"

In [107]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 4s 3ms/step - loss: 0.5747 - accuracy: 0.7164
Epoch 2/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5567 - accuracy: 0.7277
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7280
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7301
Epoch 5/150
795/804 [============================>.] - ETA: 0s - loss: 0.5478 - accuracy: 0.7319
Epoch 5: saving model to checkpoints_attempt4\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7317
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7316
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7333
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7336
Epoch 9/150
804/804 [========================

804/804 [==============================] - 2s 2ms/step - loss: 0.5288 - accuracy: 0.7416
Epoch 63/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5286 - accuracy: 0.7413
Epoch 64/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5285 - accuracy: 0.7424
Epoch 65/150
787/804 [============================>.] - ETA: 0s - loss: 0.5283 - accuracy: 0.7399
Epoch 65: saving model to checkpoints_attempt4\weights.65.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5287 - accuracy: 0.7399
Epoch 66/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5283 - accuracy: 0.7422
Epoch 67/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5286 - accuracy: 0.7415
Epoch 68/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5284 - accuracy: 0.7424
Epoch 69/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5284 - accuracy: 0.7410
Epoch 70/150
802/804 [===========================

804/804 [==============================] - 1s 2ms/step - loss: 0.5267 - accuracy: 0.7413
Epoch 125/150
798/804 [============================>.] - ETA: 0s - loss: 0.5259 - accuracy: 0.7426
Epoch 125: saving model to checkpoints_attempt4\weights.125.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5264 - accuracy: 0.7421
Epoch 126/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5265 - accuracy: 0.7424
Epoch 127/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5260 - accuracy: 0.7414
Epoch 128/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5261 - accuracy: 0.7415
Epoch 129/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5260 - accuracy: 0.7426
Epoch 130/150
783/804 [============================>.] - ETA: 0s - loss: 0.5262 - accuracy: 0.7427
Epoch 130: saving model to checkpoints_attempt4\weights.130.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5266 - accuracy: 

In [108]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5639 - accuracy: 0.7227 - 443ms/epoch - 2ms/step
Loss: 0.5639265775680542, Accuracy: 0.7226822376251221


In [109]:
# Export the model to HDF5 file
nn4.save("Attempt4.h5")